In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum
import networkx as nx
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import random

In [2]:
# LOAD DATA
#Compatibility
compatibility = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/compatibilidad_total.csv', index_col=0)
compatibility.index = range(len(compatibility))
compatibility.columns = range(len(compatibility.columns))
#Pairs, recipients and donors
pairs = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/parejas.csv',index_col=0)
recipients = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/receptores.csv',index_col=0)
donors = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/donantes.csv',index_col=0)
#Weights
hla_hr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_ar.csv",index_col=0)
hla_lr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_BR.csv",index_col=0)
hla_eplet = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_eplet.csv",index_col=0)
hla_hr.columns = hla_hr.columns.astype(int)
hla_lr.columns = hla_lr.columns.astype(int)
hla_eplet.columns = hla_eplet.columns.astype(int)

In [3]:
# Data of each locus 
base = '/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/locis_por_separado/BR/'
mats = {
    'A': 'mismatch_BR_A.csv',
    'B': 'mismatch_BR_B.csv',
    'C': 'mismatch_BR_C.csv',
    'DQ': 'mismatch_BR_DQ.csv',
    'DR': 'mismatch_BR_DR.csv'
}

matrix_mismatch = {}
for locus, archive in mats.items():
    matrix_mismatch[locus] = pd.read_csv(base + archive, index_col=0)

mismatch_ClassI = (
    matrix_mismatch['A'] +
    matrix_mismatch['B'] +
    matrix_mismatch['C']
)

mismatch_DQ = matrix_mismatch['DQ']
mismatch_DR = matrix_mismatch['DR']

In [4]:
# Mismatch to HLA score
HLA_DR = 2 - mismatch_DR
HLA_DQ = 2- mismatch_DQ
HLA_C1 = 6 - mismatch_ClassI

In [5]:
# Creation of initial graph with graph resolution and a minimum of quality (k)
def create_graph(pairs, compatibility, hla1, k):
    G = nx.DiGraph()
    added_edges = 0
    for i in pairs.index:
        for j in pairs.index:
            if compatibility.at[i, j] == 1 and hla1.at[i, j] >= k:
                G.add_edge(j, i, weight=hla1.at[i, j])
                added_edges += 1
    return G

In [6]:
# Changing initial weights to optimization weights on the initial graph arcs
def changing_resolution_weights(G, hla2):
    for u, v, data in G.edges(data=True):
        try:
            data['weight'] = hla2.iloc[int(v), int(u)]
        except KeyError:
            print(f"No se encontró peso para el arco ({u}, {v})")
        except Exception as e:
            print(f"Error al actualizar peso para el arco ({u}, {v}): {str(e)}")


In [8]:
# Optimization function 
def optimization(G, l=3, k=3):
    total_cycles = list(nx.simple_cycles(G, length_bound=3))
    valid_cycles = [cycle for cycle in total_cycles if len(cycle) <= l and all(G[u][v]['weight'] >= k for u, v in zip(cycle, cycle[1:] + cycle[:1]))]
        
    P = len(G.nodes())  # "m" of the paper
    Z = 10 # 10 for antigen and allele level. Change to 138 for eplet level optimization

    # Optimization model
    m = Model("optimization")
    m.setParam('OutputFlag', 0)

    x = {tuple(cycle): m.addVar(vtype=GRB.BINARY, name=f"x_{'_'.join(map(str, cycle))}") for cycle in valid_cycles}

    m.setObjective(
        quicksum(
            x[tuple(cycle)] * (
                (len(cycle) + (1 / P) * sum(G[u][v]['weight'] / Z for u, v in zip(cycle, cycle[1:] + cycle[:1])))
                / P
            )
            for cycle in valid_cycles
        ),
        GRB.MAXIMIZE
    )

    # Restriction
    for i in G.nodes():
        m.addConstr(quicksum(x[tuple(cycle)] for cycle in valid_cycles if i in cycle) <= 1, name=f"node_usage_{i}")

    m.optimize()

    G_optimal = nx.DiGraph()
    selected_cycles = []

    if m.status == GRB.OPTIMAL:
        for cycle in valid_cycles:
            if x[tuple(cycle)].X > 0.5:
                selected_cycles.append(cycle)
                for i in range(len(cycle)):
                    u, v = cycle[i], cycle[(i + 1) % len(cycle)]
                    G_optimal.add_edge(u, v, weight=G[u][v]['weight'])

    return G_optimal, selected_cycles

In [9]:
# Complete definition of the simulation

def update_matrices(indexes, df):
    return df.iloc[indexes, indexes]
frequiency_waiting_nodes = {}

def run_simulation(total_time, arrival_rate, departure_rate, match_run, pairs, compatibility, hla_lr, hla_hr, hla_eplet):
    # Evaluation at different resolutions
    lr_quality_ethcat1 = []
    lr_quality_ethcat2 = []
    lr_quality_ethcat4 = []
    lr_quality_ethcat5 = []
    lr_quality_ethcat6 = []
    lr_quality_ethcat7 = []
    
    hr_quality_ethcat1 = []
    hr_quality_ethcat2 = []
    hr_quality_ethcat4 = []
    hr_quality_ethcat5 = []
    hr_quality_ethcat6 = []
    hr_quality_ethcat7 = []

    e_quality_ethcat1 = []
    e_quality_ethcat2 = []
    e_quality_ethcat4 = []
    e_quality_ethcat5 = []
    e_quality_ethcat6 = []
    e_quality_ethcat7 = []

    waiting_list = []
    waiting_times = {}  
    arrivals_by_ethcat = {}
    historial_cycles = []
    historial_departures = []
    cont=0
    available_indexes = set(pairs.index)

    
    departures_by_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_classI_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DR_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DQ_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}

    # Simulation per month
    for month in range(total_time):
     
        selected_cycles = []
       
        new_entries = np.random.poisson(arrival_rate)
        new_entries = min(new_entries, len(available_indexes))
        new_index = np.random.choice(list(available_indexes), size=new_entries, replace=False)
        cont+= len(new_index)
        waiting_list.extend(new_index)
        available_indexes.difference_update(new_index)

        for idx in new_index:
            waiting_times[idx] = {'arrival': month}
            ethnicity = recipients.loc[recipients['Nodo'] == idx, 'ETHCAT'].iloc[0]
            if ethnicity in arrivals_by_ethcat:
                arrivals_by_ethcat[ethnicity] += 1
            else:
                arrivals_by_ethcat[ethnicity] = 1
        
 
        departure = np.random.poisson(departure_rate)
        departure_indexes = [] 
        if departure:
            departure = min(departure, len(waiting_list))
            departure_indexes = np.random.choice(waiting_list, size=departure, replace=False)
            waiting_list = [idx for idx in waiting_list if idx not in departure_indexes]
            historial_departures.extend(departure_indexes)

        for idx in departure_indexes:
            ethnicity = recipients.loc[idx, 'ETHCAT']
            departures_by_ethcat[ethnicity].append(idx)

        if (month + 1) % match_run == 0:
            waiting_list_index = waiting_list.copy()
            df_waiting_list = pairs.loc[waiting_list_index]
            
            filtered_compatibility = update_matrices(waiting_list_index, compatibility)
            filtered_weight = update_matrices(waiting_list_index, hla_lr) # Insert the resolution data that you want for create the graph
            
            G = create_graph(df_waiting_list, filtered_compatibility, filtered_weight, k= 3) # Insert the minimum weight for create the graph 
            changing_resolution_weights(G, hla_lr) # Insert the resolution data that you want for the optimization
            
            G_optimal, selected_cycles = optimization(G, l=3, k=3) # Insert the minimum quality that you want for the cycles 

            for u, v, data in G_optimal.edges(data=True):
                node_ethcat = recipients.loc[v, 'ETHCAT'] 
                
                # HLA for differents loci (optimization resolution)
                value_HLA_classI = HLA_C1.iloc[v, u]
                value_HLA_DR = HLA_DR.iloc[v, u]
                value_HLA_DQ = HLA_DQ.iloc[v, u]

                HLA_classI_ethcat[node_ethcat].append(value_HLA_classI)
                HLA_DR_ethcat[node_ethcat].append(value_HLA_DR)
                HLA_DQ_ethcat[node_ethcat].append(value_HLA_DQ)


                if node_ethcat == 1:
                    lr_quality_ethcat1.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat1.append(hla_hr.iloc[v, u])
                    e_quality_ethcat1.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 2:
                    lr_quality_ethcat2.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat2.append(hla_hr.iloc[v, u])
                    e_quality_ethcat2.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 4:
                    lr_quality_ethcat4.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat4.append(hla_hr.iloc[v, u])
                    e_quality_ethcat4.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 5:
                    lr_quality_ethcat5.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat5.append(hla_hr.iloc[v, u])
                    e_quality_ethcat5.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 6:
                    lr_quality_ethcat6.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat6.append(hla_hr.iloc[v, u])
                    e_quality_ethcat6.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 7:
                    lr_quality_ethcat7.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat7.append(hla_hr.iloc[v, u])
                    e_quality_ethcat7.append(hla_eplet.iloc[v, u])
            historial_cycles.extend(selected_cycles)

            nodes_in_cycles = [node for cycle in selected_cycles for node in cycle]
            waiting_list = [idx for idx in waiting_list if idx not in nodes_in_cycles]
            for idx in nodes_in_cycles:
                waiting_times[idx]['departure'] = month

    nodes_in_historial = [node for cycle in historial_cycles for node in cycle]

    data = []

    for idx in nodes_in_historial:
        if idx in waiting_times:
            waiting_time = waiting_times[idx]['departure'] - waiting_times[idx]['arrival']
            ethnicity = recipients.loc[idx, 'ETHCAT'] 
            data.append({'node': idx, 'waiting time (months)': waiting_time, 'ethnicity': ethnicity})

    

    df_ethnicity = pd.DataFrame(data)
    df_ethnicity['ethnicity'] = df_ethnicity['ethnicity'].astype(int)
    df_ethnicity1 = df_ethnicity[df_ethnicity['ethnicity'] == 1]
    df_ethnicity2 = df_ethnicity[df_ethnicity['ethnicity'] == 2]
    df_ethnicity4 = df_ethnicity[df_ethnicity['ethnicity'] == 4]
    df_ethnicity5 = df_ethnicity[df_ethnicity['ethnicity'] == 5]
    df_ethnicity6 = df_ethnicity[df_ethnicity['ethnicity'] == 6]
    df_ethnicity7 = df_ethnicity[df_ethnicity['ethnicity'] == 7]
    waiting_time_mean_1 = df_ethnicity1['waiting time (months)'].mean()
    waiting_time_mean_2 = df_ethnicity2['waiting time (months)'].mean()
    waiting_time_mean_4 = df_ethnicity4['waiting time (months)'].mean()
    waiting_time_mean_5 = df_ethnicity5['waiting time (months)'].mean()
    waiting_time_mean_6 = df_ethnicity6['waiting time (months)'].mean()
    waiting_time_mean_7 = df_ethnicity7['waiting time (months)'].mean()

    
    outgoing_per_ethcat = {e: len(departures_by_ethcat[e]) for e in [1,2,4,5,6,7]}
    L_per_ethcat  = {e: outgoing_per_ethcat [e] / arrivals_by_ethcat.get(e, 1) for e in outgoing_per_ethcat } 
    F_per_ethcat  = {e: len([c for c in historial_cycles for n in c if recipients.loc[n,'ETHCAT']==e]) / arrivals_by_ethcat.get(e, 1)
                   for e in [1,2,4,5,6,7]}  
    
     
    total_trasplants = sum(len(cycle) for cycle in historial_cycles)
    total_entries = sum(arrivals_by_ethcat.values())
    F_total = total_trasplants / total_entries if total_entries > 0 else 0


     

    return {
        'historial_cycles': historial_cycles,
        'historial_departures': historial_departures,
        'entries_nodes': cont,
        'arrivals_by_ethcat': arrivals_by_ethcat,
        'waiting_time_mean1':   waiting_time_mean_1 ,
        'waiting_time_mean2':   waiting_time_mean_2,
        'waiting_time_mean4':   waiting_time_mean_4,
        'waiting_time_mean5':   waiting_time_mean_5,
        'waiting_time_mean6':   waiting_time_mean_6,
        'waiting_time_mean7':   waiting_time_mean_7,
        'lr_quality_ethcat1': np.mean(lr_quality_ethcat1),
        'lr_quality_ethcat2': np.mean(lr_quality_ethcat2),
        'lr_quality_ethcat4': np.mean(lr_quality_ethcat4),
        'lr_quality_ethcat5': np.mean(lr_quality_ethcat5),
        'lr_quality_ethcat6': np.mean(lr_quality_ethcat6),
        'lr_quality_ethcat7': np.mean(lr_quality_ethcat7),

        'hr_quality_ethcat1': np.mean(hr_quality_ethcat1),
        'hr_quality_ethcat2': np.mean(hr_quality_ethcat2),
        'hr_quality_ethcat4': np.mean(hr_quality_ethcat4),
        'hr_quality_ethcat5': np.mean(hr_quality_ethcat5),
        'hr_quality_ethcat6': np.mean(hr_quality_ethcat6),
        'hr_quality_ethcat7': np.mean(hr_quality_ethcat7),

        'e_quality_ethcat1': np.mean(e_quality_ethcat1),
        'e_quality_ethcat2': np.mean(e_quality_ethcat2),
        'e_quality_ethcat4': np.mean(e_quality_ethcat4),
        'e_quality_ethcat5': np.mean(e_quality_ethcat5),
        'e_quality_ethcat6': np.mean(e_quality_ethcat6),
        'e_quality_ethcat7': np.mean(e_quality_ethcat7),
        'avg_ClassI_HLA_ethcat1': np.mean(HLA_classI_ethcat[1]), 
        'avg_ClassI_HLA_ethcat2': np.mean(HLA_classI_ethcat[2]),
        'avg_ClassI_HLA_ethcat4': np.mean(HLA_classI_ethcat[4]),
        'avg_ClassI_HLA_ethcat5': np.mean(HLA_classI_ethcat[5]),
        'avg_ClassI_HLA_ethcat6': np.mean(HLA_classI_ethcat[6]),
        'avg_ClassI_HLA_ethcat7': np.mean(HLA_classI_ethcat[7]), 
        'HLA_ClassI_total': np.mean(
                                    HLA_classI_ethcat[1] + 
                                    HLA_classI_ethcat[2] + 
                                    HLA_classI_ethcat[4] + 
                                    HLA_classI_ethcat[5] + 
                                    HLA_classI_ethcat[6] + 
                                    HLA_classI_ethcat[7]
                                ),

        'avg_DR_HLA_ethcat1': np.mean(HLA_DR_ethcat[1]), 
        'avg_DR_HLA_ethcat2': np.mean(HLA_DR_ethcat[2]),
        'avg_DR_HLA_ethcat4': np.mean(HLA_DR_ethcat[4]),
        'avg_DR_HLA_ethcat5': np.mean(HLA_DR_ethcat[5]),
        'avg_DR_HLA_ethcat6': np.mean(HLA_DR_ethcat[6]),
        'avg_DR_HLA_ethcat7': np.mean(HLA_DR_ethcat[7]), 
        'HLA_DR_total': np.mean(
                                    HLA_DR_ethcat[1] + 
                                    HLA_DR_ethcat[2] + 
                                    HLA_DR_ethcat[4] + 
                                    HLA_DR_ethcat[5] + 
                                    HLA_DR_ethcat[6] + 
                                    HLA_DR_ethcat[7]
                                ),

        'avg_DQ_HLA_ethcat1': np.mean(HLA_DQ_ethcat[1]), 
        'avg_DQ_HLA_ethcat2': np.mean(HLA_DQ_ethcat[2]),
        'avg_DQ_HLA_ethcat4': np.mean(HLA_DQ_ethcat[4]),
        'avg_DQ_HLA_ethcat5': np.mean(HLA_DQ_ethcat[5]),
        'avg_DQ_HLA_ethcat6': np.mean(HLA_DQ_ethcat[6]),
        'avg_DQ_HLA_ethcat7': np.mean(HLA_DQ_ethcat[7]), 
        'HLA_DQ_total': np.mean(
                                    HLA_DQ_ethcat[1] + 
                                    HLA_DQ_ethcat[2] + 
                                    HLA_DQ_ethcat[4] + 
                                    HLA_DQ_ethcat[5] + 
                                    HLA_DQ_ethcat[6] + 
                                    HLA_DQ_ethcat[7]
                                ),

        'F_per_ethcat': F_per_ethcat,        
        'L_per_ethcat' : L_per_ethcat,          
        'F_total': F_total,
        'lr_quality_total': np.mean(
                                    lr_quality_ethcat1 + 
                                    lr_quality_ethcat2 + 
                                    lr_quality_ethcat4 + 
                                    lr_quality_ethcat5 + 
                                    lr_quality_ethcat6 + 
                                    lr_quality_ethcat7
                                ),
        'hr_quality_total': np.mean(
                                    hr_quality_ethcat1 + 
                                    hr_quality_ethcat2 + 
                                    hr_quality_ethcat4 + 
                                    hr_quality_ethcat5 + 
                                    hr_quality_ethcat6 + 
                                    hr_quality_ethcat7
                                ),
        'e_quality_total': np.mean(
                                    e_quality_ethcat1 + 
                                    e_quality_ethcat2 + 
                                    e_quality_ethcat4 + 
                                    e_quality_ethcat5 + 
                                    e_quality_ethcat6 + 
                                    e_quality_ethcat7
                                ),
        'Waiting_time_mean': pd.concat([
                                    df_ethnicity1['waiting time (months)'],
                                    df_ethnicity2['waiting time (months)'],
                                    df_ethnicity4['waiting time (months)'],
                                    df_ethnicity5['waiting time (months)'],
                                    df_ethnicity6['waiting time (months)'],
                                    df_ethnicity7['waiting time (months)']
                                    ]).mean()


        }

In [ ]:
# Runing the simulations 

simulations_results = []
for i in range(100):
    print(f"Runing simulation {i + 1} of 100")
    results = run_simulation(
        total_time=10*12, 
        arrival_rate=(990/(10*12)), 
        departure_rate=((990/(10*12))*0.29), 
        match_run=3,
        pairs=pairs, 
        compatibility=compatibility, 
        hla_lr = hla_lr,
        hla_hr=hla_hr,
        hla_eplet= hla_eplet
    )
   
    simulations_results.append({
        'simulation': i + 1,
        'cycles': results['historial_cycles'],
        'total_cycles': len(results['historial_cycles']),
        'total_departures': len(results['historial_departures']),
        'total_entries': results['entries_nodes'],
        'arrivals_by_ethcat': results['arrivals_by_ethcat'],
        'waiting_time_mean1':   results['waiting_time_mean1'] ,
        'waiting_time_mean2':   results['waiting_time_mean2'],
        'waiting_time_mean4':   results['waiting_time_mean4'],
        'waiting_time_mean5':   results['waiting_time_mean5'],
        'waiting_time_mean6':   results['waiting_time_mean6'],
        'waiting_time_mean7':   results['waiting_time_mean7'],
        'lr_quality_ethcat1': results['lr_quality_ethcat1'],
        'lr_quality_ethcat2': results['lr_quality_ethcat2'],
        'lr_quality_ethcat4': results['lr_quality_ethcat4'],
        'lr_quality_ethcat5': results['lr_quality_ethcat5'],
        'lr_quality_ethcat6': results['lr_quality_ethcat6'],
        'lr_quality_ethcat7': results['lr_quality_ethcat7'],
        'hr_quality_ethcat1': results['hr_quality_ethcat1'],
        'hr_quality_ethcat2': results['hr_quality_ethcat2'],
        'hr_quality_ethcat4': results['hr_quality_ethcat4'],
        'hr_quality_ethcat5': results['hr_quality_ethcat5'],
        'hr_quality_ethcat6': results['hr_quality_ethcat6'],
        'hr_quality_ethcat7': results['hr_quality_ethcat7'],
        'e_quality_ethcat1': results['e_quality_ethcat1'],
        'e_quality_ethcat2': results['e_quality_ethcat2'],
        'e_quality_ethcat4': results['e_quality_ethcat4'],
        'e_quality_ethcat5': results['e_quality_ethcat5'],
        'e_quality_ethcat6': results['e_quality_ethcat6'],
        'e_quality_ethcat7': results['e_quality_ethcat7'],
        'avg_ClassI_HLA_ethcat1': results['avg_ClassI_HLA_ethcat1'],
        'avg_ClassI_HLA_ethcat2': results['avg_ClassI_HLA_ethcat2'],
        'avg_ClassI_HLA_ethcat4': results['avg_ClassI_HLA_ethcat4'],
        'avg_ClassI_HLA_ethcat5': results['avg_ClassI_HLA_ethcat5'],
        'avg_ClassI_HLA_ethcat6': results['avg_ClassI_HLA_ethcat6'],
        'avg_ClassI_HLA_ethcat7': results['avg_ClassI_HLA_ethcat7'],
        'HLA_ClassI_total': results['HLA_ClassI_total'],

        'avg_DR_HLA_ethcat1': results['avg_DR_HLA_ethcat1'],
        'avg_DR_HLA_ethcat2': results['avg_DR_HLA_ethcat2'],
        'avg_DR_HLA_ethcat4': results['avg_DR_HLA_ethcat4'],
        'avg_DR_HLA_ethcat5': results['avg_DR_HLA_ethcat5'],
        'avg_DR_HLA_ethcat6': results['avg_DR_HLA_ethcat6'],
        'avg_DR_HLA_ethcat7': results['avg_DR_HLA_ethcat7'],
        'HLA_DR_total': results['HLA_DR_total'],
        

        'avg_DQ_HLA_ethcat1': results['avg_DQ_HLA_ethcat1'],
        'avg_DQ_HLA_ethcat2': results['avg_DQ_HLA_ethcat2'],
        'avg_DQ_HLA_ethcat4': results['avg_DQ_HLA_ethcat4'],
        'avg_DQ_HLA_ethcat5': results['avg_DQ_HLA_ethcat5'],
        'avg_DQ_HLA_ethcat6': results['avg_DQ_HLA_ethcat6'],
        'avg_DQ_HLA_ethcat7': results['avg_DQ_HLA_ethcat7'],
        'HLA_DQ_total': results['HLA_DQ_total'],

        'F_per_ethcat':   results['F_per_ethcat'],   
        'L_per_ethcat':   results['L_per_ethcat'],   
        'F_total': results['F_total'],
        'lr_quality_total': results['lr_quality_total'],
        'hr_quality_total': results['hr_quality_total'],
        'e_quality_total': results['e_quality_total'],
        'Waiting_time_mean': results['Waiting_time_mean']
    })
    

Runing simulation 1 of 100
Restricted license - for non-production use only - expires 2026-11-23
Runing simulation 2 of 100
Runing simulation 3 of 100
Runing simulation 4 of 100
Runing simulation 5 of 100
Runing simulation 6 of 100
Runing simulation 7 of 100
Runing simulation 8 of 100
Runing simulation 9 of 100
Runing simulation 10 of 100
Runing simulation 11 of 100
Runing simulation 12 of 100
Runing simulation 13 of 100
Runing simulation 14 of 100
Runing simulation 15 of 100
Runing simulation 16 of 100
Runing simulation 17 of 100
Runing simulation 18 of 100
Runing simulation 19 of 100
Runing simulation 20 of 100
Runing simulation 21 of 100
Runing simulation 22 of 100
Runing simulation 23 of 100
Runing simulation 24 of 100
Runing simulation 25 of 100
Runing simulation 26 of 100
Runing simulation 27 of 100
Runing simulation 28 of 100
Runing simulation 29 of 100
Runing simulation 30 of 100
Runing simulation 31 of 100
Runing simulation 32 of 100
Runing simulation 33 of 100
Runing simulati

In [20]:
df_results = pd.DataFrame(simulations_results)

list_HLA_ClassI_total = df_results['HLA_ClassI_total'].tolist()


list_HLA_DR_total = df_results['HLA_DR_total'].tolist()


list_HLA_DQ_total = df_results['HLA_DQ_total'].tolist()



lists_ClassI = {
    1: df_results['avg_ClassI_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_ClassI_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_ClassI_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_ClassI_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_ClassI_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_ClassI_HLA_ethcat7'].dropna().tolist()
}

lists_DR = {
    1: df_results['avg_DR_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DR_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DR_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DR_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DR_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DR_HLA_ethcat7'].dropna().tolist()
}

lists_DQ = {
    1: df_results['avg_DQ_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DQ_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DQ_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DQ_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DQ_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DQ_HLA_ethcat7'].dropna().tolist()
}



In [21]:
import scipy.stats as st

ethnicity = [1, 2, 4, 5, 6, 7]
n_sim = len(df_results)

def media_ci(list):
    m = np.mean(list)
    s = np.std(list, ddof=1)
    low, high = st.t.interval(0.95, len(list)-1, loc=m, scale=s/np.sqrt(len(list)))
    return m, f"{m:.3f} [{low:.3f}; {high:.3f}]"

rows = []
# Reporting per ethnicity 

for e in ethnicity:
    arrivals_e = df_results['arrivals_by_ethcat'].apply(lambda d: d.get(e, 0)).sum() / n_sim
    transplants_e = df_results['cycles'].apply(
        lambda cycles: sum(1 for c in cycles for n in c if recipients.loc[n, 'ETHCAT'] == e)
    ).sum() / n_sim

    F_vals = df_results['F_per_ethcat'].apply(lambda d: d[e])
    _, txt_F = media_ci(F_vals)
    std_F = np.std(F_vals, ddof=1)

    lr_quality = df_results[f'lr_quality_ethcat{e}']
    _, txt_antigen = media_ci(lr_quality)

    hr_quality = df_results[f'hr_quality_ethcat{e}']
    _, txt_allele = media_ci(hr_quality)

    e_quality  = df_results[f'e_quality_ethcat{e}']
    _, txt_eplet = media_ci(e_quality)

    waiting = df_results[f'waiting_time_mean{e}'].mean()

    L_vals = df_results['L_per_ethcat'].apply(lambda d: d[e])
    _, txt_L = media_ci(L_vals)

    f_mean = np.mean(F_vals)
    l_mean = np.mean(L_vals)
    still_in_kep = round(1 - f_mean - l_mean, 3)


    list_classi = lists_ClassI[e]

    _, txt_classi = media_ci(list_classi)

    list_dr = lists_DR[e]
    _, txt_dr = media_ci(list_dr)

    list_dq = lists_DQ[e]
    _, txt_dq = media_ci(list_dq)


    rows.append({
        'Ethnicity(s)': e,
        'Arrivals': round(arrivals_e, 2),
        'Transplants': round(transplants_e, 2),
        'F(s) (Matched)': txt_F,
        'HLA(s) Antigen': txt_antigen,
        'HLA(s) Allele': txt_allele,
        'HLA(s) Eplets': txt_eplet,
        'Waiting Time': round(waiting, 3),
        'L(s) (Left Unmatched)': txt_L,
        '1-F(s)-L(s) (Still in KEP)': still_in_kep,
        'HLA ClassI': txt_classi,
        'HLA DR': txt_dr,
        'HLA DQ': txt_dq,

    })

# Reporting the totals

total_arrivals = sum(df_results['arrivals_by_ethcat'].apply(lambda d: sum(d.values()))) / n_sim
total_transplants = df_results['cycles'].apply(lambda cycles: sum(len(c) for c in cycles)).sum() / n_sim
F_total = total_transplants / total_arrivals


F_s_total, txt_F_total = media_ci(df_results['F_total'])

L_vals_flat = (df_results['total_departures'] / df_results['total_entries']).tolist()
L_s_total, ic_L_total = media_ci(L_vals_flat)
txt_L_total = ic_L_total

_, txt_total_lr_quality = media_ci(df_results['lr_quality_total'])
_,txt_total_hr_quality = media_ci(df_results['hr_quality_total'])
_,txt_total_e_quality= media_ci(df_results['e_quality_total'])

txt_waiting_time_mean = np.mean(df_results['Waiting_time_mean'])

F_vals_flat = [df_results['F_per_ethcat'].iloc[i][e] for i in range(n_sim) for e in ethnicity]
std_F_total = np.std(F_vals_flat, ddof=1)

f_total_mean = F_total  
total_outgoing = sum(
    df_results['L_per_ethcat'].apply(lambda d: sum(d.values()))
) / n_sim

l_total_mean = total_outgoing / total_arrivals
still_in_kep_total = round(1 - F_total - l_total_mean, 3)


_, txt_classi_total = media_ci(list_HLA_ClassI_total)

_, txt_dr_total = media_ci(list_HLA_DR_total )

_, txt_dq_total = media_ci(list_HLA_DQ_total )

rows.append({
    'Ethnicity(s)': 'Entire Population',
    'Arrivals': round(total_arrivals, 2),
    'Transplants': round(total_transplants, 2),
    'F(s) (Matched)': txt_F_total,
    'HLA(s) Antigen': txt_total_lr_quality,
    'HLA(s) Allele': txt_total_hr_quality,
    'HLA(s) Eplets': txt_total_e_quality,
    'Waiting Time': txt_waiting_time_mean,
    'L(s) (Left Unmatched)': txt_L_total,
    '1-F(s)-L(s) (Still in KEP)': f"{(1 - L_s_total - F_s_total):.3f}",
    'HLA ClassI': txt_classi_total,
    'HLA DR': txt_dr_total,
    'HLA DQ': txt_dq_total
})

df_final_table = pd.DataFrame(rows)
display(df_final_table)

,Ethnicity(s),Arrivals,Transplants,F(s) (Matched),HLA(s) Antigen,HLA(s) Allele,HLA(s) Eplets,Waiting Time,L(s) (Left Unmatched),1-F(s)-L(s) (Still in KEP),HLA ClassI,HLA DR,HLA DQ
0,1,591.60,412.78,0.698 [0.693; 0.702],4.694 [4.681; 4.708],3.383 [3.368; 3.399],93.096 [92.975; 93.217],2.90700,0.271 [0.267; 0.275],0.031,2.561 [2.551; 2.571],0.944 [0.939; 0.949],1.189 [1.185; 1.194]
1,2,169.95,107.86,0.635 [0.627; 0.642],4.151 [4.130; 4.171],2.544 [2.523; 2.565],90.804 [90.600; 91.008],3.16800,0.326 [0.319; 0.334],0.039,2.131 [2.115; 2.148],0.848 [0.837; 0.859],1.171 [1.161; 1.181]
2,4,147.33,91.62,0.622 [0.614; 0.629],4.301 [4.272; 4.329],2.656 [2.629; 2.683],91.045 [90.793; 91.298],3.51500,0.334 [0.326; 0.343],0.044,2.330 [2.304; 2.357],0.852 [0.841; 0.863],1.119 [1.107; 1.130]
3,5,55.65,34.24,0.615 [0.603; 0.628],3.959 [3.920; 3.999],2.076 [2.037; 2.116],85.063 [84.582; 85.545],3.67300,0.339 [0.327; 0.352],0.045,2.142 [2.111; 2.173],0.739 [0.719; 0.759],1.078 [1.058; 1.098]
4,6,7.92,6.09,0.769 [0.745; 0.793],4.314 [4.229; 4.399],2.976 [2.853; 3.099],85.085 [84.224; 85.946],2.10600,0.201 [0.179; 0.224],0.03,2.253 [2.175; 2.332],0.901 [0.849; 0.952],1.160 [1.117; 1.203]
5,7,5.97,2.88,0.483 [0.449; 0.517],4.028 [3.905; 4.151],2.234 [2.076; 2.392],93.194 [91.848; 94.541],4.61000,0.448 [0.410; 0.486],0.069,2.106 [1.990; 2.222],0.758 [0.688; 0.829],1.163 [1.088; 1.239]
6,Entire Population,978.42,655.47,0.670 [0.666; 0.674],4.505 [4.494; 4.516],3.066 [3.054; 3.078],91.935 [91.840; 92.031],3.08009,0.295 [0.291; 0.299],0.035,2.431 [2.423; 2.440],0.903 [0.899; 0.907],1.170 [1.166; 1.174]


In [ ]:
df_final_table.to_excel("ofresults_BRBR.xlsx", index=True)